**Step 1: Find all patients with condition X from three sources**

**Required**
* Connect to FHIR Synthetic Patient Data
* Connect to OHDSI Synthetic Patient Data
* Connect to HUSH+ Synthetic Patient Data

**Step 1.1: Find 

In [ ]:
## All the imports we need
from urllib2 import Request, urlopen, URLError
from urllib import quote_plus

import mysql.connector

import pprint
import requests

from datetime import datetime, timedelta
from dateutil.parser import parse as parse_date
from greentranslator.api import GreenTranslator


In [7]:
#connect to UMLS
try:
  cnx = mysql.connector.connect(user='tadmin',
                                password='ncats_translator!',
                                database='umls',
                                host='translator.ceyknq0yekb3.us-east-1.rds.amazonaws.com')
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
else:
  print
  #cnx.close()

In [3]:
#Define functions
def findMeshAndICD10(name):
    cursor = cnx.cursor()
    query = ("SELECT CUI FROM umls.MRCONSO WHERE STR='"+name+"'")
    cursor.execute(query, ())
    res = "Undef"
    for code in cursor:
        if res=="Undef":
            res = code
    if res != "Undef":
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='MSH' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        msh = "Undef"
        for code in cursor:
            msh = code
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD10' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        icd10 = "Undef"
        for code in cursor:
            icd10 = code
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD10CM' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        for code in cursor:
            icd10 = code
        return (msh[0],icd10[0])
    return ("Undef","Undef") 


In [4]:
findMeshAndICD10('Asthma')

(u'D001249', u'J45.90')

In [ ]:
## FHIR patient data

base_url = 'http://ictrweb.johnshopkins.edu/rest/synthetic'
patients = base_url+"/Patient"+"?_count=50"

req = requests.get(patients)

In [62]:
## Given disease/condition term, get back ICD codes
url_con = "http://api.ohdsi.org/WebAPI/vocabulary/search"
headers = {'content-type': 'application/json'}
params = {"QUERY": "Asthma",
          "VOCABULARY_ID": ["ICD9CM"]
}
response = requests.post(url_con, data=json.dumps(params), headers=headers)
data= json.loads(response.text.decode('utf-8'))
icd9arr=[]
for d in data:
    icd9arr.append(d["CONCEPT_CODE"])
print icd9arr

[u'E945.7', u'493', u'493.9', u'493.90', u'493.92', u'493.91', u'493.2', u'493.20', u'493.22', u'493.21', u'493.82', u'493.0', u'493.00', u'493.02', u'493.01', u'V17.5', u'493.1', u'493.10', u'493.12', u'493.11', u'493.8', u'975.7']


In [8]:
## Pull patients matching a list of ICD codes
query = GreenTranslator ().get_query ()

patients = query.clinical_get_patients (age='8', sex='male', 
                                        race='white', location='OUTPATIENT')
pprint.pprint (patients)

[{u'birth_date': u'2006-08-02 00:00:00',
  u'diag': {u'ICD10:B08.4': {u'2016-07-08 00:00:00': u'OUTPATIENT'},
            u'ICD10:E66.09': {u'2016-05-04 00:00:00': u'OUTPATIENT'},
            u'ICD10:F41.1': {u'2017-02-28 00:00:00': u'OUTPATIENT'},
            u'ICD10:F41.9': {u'2016-03-10 00:00:00': u'OUTPATIENT'},
            u'ICD10:F42.4': {u'2015-10-28 00:00:00': u'OUTPATIENT'},
            u'ICD10:F43.0': {u'2017-03-15 00:00:00': u'OUTPATIENT'},
            u'ICD10:F45.9': {u'2016-05-04 00:00:00': u'OUTPATIENT'},
            u'ICD10:F84.0': {u'2017-03-15 00:00:00': u'OUTPATIENT'},
            u'ICD10:F91.9': {u'2015-11-18 00:00:00': u'OUTPATIENT'},
            u'ICD10:H60.502': {u'2016-06-29 00:00:00': u'OUTPATIENT'},
            u'ICD10:H91.90': {u'2017-03-15 00:00:00': u'OUTPATIENT'},
            u'ICD10:H91.93': {u'2016-05-04 00:00:00': u'OUTPATIENT'},
            u'ICD10:J02.9': {u'2017-01-20 00:00:00': u'OUTPATIENT'},
            u'ICD10:J06.9': {u'2015-12-17 00:00:00': u'OU

            u'ICD10:Q16.5': {u'2017-03-19 00:00:00': u'OUTPATIENT'},
            u'ICD10:R05': {u'2016-05-21 00:00:00': u'OUTPATIENT'},
            u'ICD10:R09.82': {u'2015-12-11 00:00:00': u'OUTPATIENT'},
            u'ICD10:R22.0': {u'2015-12-11 00:00:00': u'OUTPATIENT'},
            u'ICD10:R74.8': {u'2017-03-19 00:00:00': u'OUTPATIENT'},
            u'ICD10:T50.995A': {u'2016-06-26 00:00:00': u'OUTPATIENT'},
            u'ICD10:T78.2XXA': {u'2017-03-18 00:00:00': u'OUTPATIENT'},
            u'ICD10:T78.3XXA': {u'2016-06-26 00:00:00': u'OUTPATIENT'},
            u'ICD10:T88.6XXA': {u'2016-06-26 00:00:00': u'OUTPATIENT'},
            u'ICD10:Z01.818': {u'2016-07-17 00:00:00': u'OUTPATIENT'},
            u'ICD10:Z09': {u'2016-10-22 00:00:00': u'OUTPATIENT'},
            u'ICD10:Z41.8': {u'2017-03-18 00:00:00': u'OUTPATIENT'},
            u'ICD10:Z46.1': {u'2016-12-26 00:00:00': u'OUTPATIENT'},
            u'ICD10:Z79.2': {u'2016-06-24 00:00:00': u'OUTPATIENT'},
            u'ICD10:Z79

In [9]:
meds = {}
for x in patients:
    medList = x['medList']
    # Collect the unique meds
    for m in medList.keys():
        found = False
        try:
            meds[medList[m]] = meds[medList[m]]+1
        except KeyError:
            meds[medList[m]] = 1
pprint.pprint(meds)

{None: 9,
 u'0.3 ML Epinephrine 1 MG/ML Auto-Injector [Epipen]': 1,
 u'120 ACTUAT Fluticasone propionate 0.05 MG/ACTUAT Nasal Inhaler': 2,
 u'24 HR Methylphenidate Hydrochloride 10 MG Extended Release Oral Capsule [Ritalin]': 1,
 u'ACYCLOVIR 5 % TOPICAL OINTMENT': 1,
 u'ADVAIR HFA 230-21 MCG INHALER': 1,
 u'ALBUTEROL 90 MCG INHALER': 1,
 u'ALBUTEROL SULF HFA 90 MCG INH': 7,
 u'ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) SOLUTION FOR NEBULIZATION': 9,
 u'ALBUTEROL SULFATE HFA 90 MCG/ACTUATION AEROSOL INHALER': 18,
 u'AMOXICILLIN 250 MG CHEWABLE TABLET': 1,
 u'AMOXICILLIN 875 MG TABLET': 1,
 u'AUGMENTIN 400-57 MG/5 ML': 1,
 u'AUGMENTIN ES-600 SUSPENSION': 3,
 u'AZITHROMYCIN 250 MG TABLET': 11,
 u'Acetaminophen 21.7 MG/ML / Hydrocodone Bitartrate 0.5 MG/ML Oral Solution': 1,
 u'Acetaminophen 32 MG/ML Oral Suspension': 1,
 u'Albuterol 0.83 MG/ML Inhalant Solution': 3,
 u'Amitriptyline Hydrochloride 10 MG Oral Tablet': 1,
 u'Amoxicillin 120 MG/ML / Clavulanate 8.58 MG/ML Oral Suspension': 4,
 u

In [67]:
## For a given medication string
def med2rxnorm(txt):
    url = 'http://data.bioontology.org/annotator?text=%s&apikey=b792dd1b-cdc2-4cc8-aaf2-4fa4fbf47e4e'
    txt = urlopen(url % quote_plus(txt)).read()
    resp = json.loads(txt)
    if len(resp) == 0: return(None)
    return([ x['text'] for x in resp[0]['annotations'] ])
print med2rxnorm("Azithromycin 20 mg twice daily")


[u'AZITHROMYCIN']
